# GPT-J-6B Inference Demo

This notebook demonstrates how to run the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

## Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2185 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/

In [ ]:
!apt-get install zstd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 66 not upgraded.
Need to get 278 kB of archives.
After this operation, 1141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 0s (591 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package zstd.
(Reading database ... 17282 files and directories currently installed.)
Preparing to unpack .../zstd_1.3.3+dfsg-2ubuntu1.2_amd64.deb ...
Unpacking zstd (1.3.3+dfsg-2ubuntu1.2) ...
Setting up zstd (1.3.3+dfsg-2ubuntu1.2) ...


In [ ]:
!apt-get install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  wget
0 upgraded, 1 newly installed, 0 to remove and 66 not upgraded.
Need to get 316 kB of archives.
After this operation, 954 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 wget amd64 1.19.4-1ubuntu2.2 [316 kB]
Fetched 316 kB in 0s (685 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package wget.
(Reading database ... 17299 files and directories currently installed.)
Preparing to unpack .../wget_1.19.4-1ubuntu2.2_amd64.deb ...
Unpacking wget (1.19.4-1ubuntu2.2) ...
Setting up wget (1.19.4-1ubuntu2.2) ...


In [ ]:
# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!wget https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

--2021-06-09 16:41:47--  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9414712325 (8.8G) [application/octet-stream]
Saving to: ‘step_383500_slim.tar.zstd’

step_383500_slim.ta  16%[==>                 ]   1.43G  2.37MB/s    in 2m 1s   

2021-06-09 16:43:48 (12.1 MB/s) - Read error at byte 1531576061/9414712325 (Connection reset by peer). Retrying.

--2021-06-09 16:43:49--  (try: 2)  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 9414712325 (8.8G), 7883136264 (7.3G) remaining [application/octet-stream]
Saving to: ‘step_383500_slim.tar.zstd’

step_383500_slim.ta  68%[+++=========>       ]   5.98G  2.62MB/s    in 8m 35s  

2021-06-09 16:5

In [ ]:
!tar -I zstd -xf step_383500_slim.tar.zstd

In [ ]:
!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

Cloning into 'mesh-transformer-jax'...
remote: Enumerating objects: 475, done.
remote: Counting objects: 100% (475/475), done.
remote: Compressing objects: 100% (319/319), done.
remote: Total 475 (delta 310), reused 303 (delta 141), pack-reused 0
Receiving objects: 100% (475/475), 135.64 KiB | 7.98 MiB/s, done.
Resolving deltas: 100% (310/310), done.
  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-10co2dsu
  Cloning https://github.com/EleutherAI/lm-evaluation-harness/ to /tmp/pip-req-build-ksyun7sz
     |████████████████████████████████| 2.0 MB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 28.6 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 96 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 47.5 MB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 630 kB 29.7 MB/s eta 0:

     |████████████████████████████████| 131 kB 38.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 166 kB 33.8 MB/s eta 0:00:01
     |████████████████████████████████| 291 kB 31.7 MB/s eta 0:00:01
     |████████████████████████████████| 206 kB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 29.5 MB/s eta 0:00:01
     |████████████████████████████████| 45.3 MB 242 kB/s  eta 0:00:01     |███████████████████████████▎    | 38.6 MB 24.3 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 201 kB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 272 kB 34.5 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 41.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 38.1 MB/s eta 0:00:01
     |██████████████████████████████

     |████████████████████████████████| 743 kB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 31.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 36.9 MB/s eta 0:00:01     |█                               | 307 kB 36.9 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 14.8 MB/s ta 0:00:01
     |████████████████████████████████| 81 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 20.7 MB 321 kB/s  eta 0:00:01
     |████████████████████████████████| 243 kB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 179 kB 33.2 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 63 kB 3.4 MB/s  eta 0:00:01
     |████████████████████████████████| 961 kB 30.4 MB/s eta 0:00:01
     |███████████████████████████████

     |████████████████████████████████| 198 kB 31.9 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 12.5 MB/s ta 0:00:01
  Created wheel for jax: filename=jax-0.2.13-py3-none-any.whl size=730503 sha256=8845b39bd13bf06e02d27fab0043cabb3d1c4af4db72e4faa6cbff7df06d64f8
  Stored in directory: /root/.cache/pip/wheels/5b/ec/f9/a631f8f6e108545c5ee8b7ea9dfde782356040c2677aa9bf3e
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15077 sha256=da5a5c52b892965a74a4962437372ab53d8715fda90fd494dba89bfeaf1caf7b
  Stored in directory: /root/.cache/pip/wheels/f5/f7/77/59440b4bf3fb41755539a8891b0e01164c4ff597ed2c1742fb
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41913 sha256=aceed8d78e09b2dc68411b5b2c8df00dea86cd13a451e1375ead6fd1840ea833
  Stored in directory: /root/.cache/pip/wheels/ff/2a/24/75041425faf3347ab146a4a3d0484f723b2c44a7966a06e3f0
  Created wheel for dm-haiku: filename=dm_haiku-0.0.5.dev0-py3-none-any.whl size=296281 

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-api-core 1.30.0 requires google-auth<2.0dev,>=1.25.0, but you'll have google-auth 1.24.0 which is incompatible.
flask 1.1.4 requires click<8.0,>=5.1, but you'll have click 8.0.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Processing ./mesh-transformer-jax
     |████████████████████████████████| 590 kB 27.0 MB/s eta 0:00:01
  Created wheel for jax: filename=jax-0.2.12-py3-none-any.whl size=682484 sha256=feebc482f4c9c55571262b332f53b3e849177d5689c1f915b48e0e39a1dd9a02
  Stored in directory: /root/.cache/pip/wheels/29/32/f2/88b767ba229d3fc80c4775d623a7353afdba602cc9ff9051f2
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-py3-none-any.whl size=19899 sha256=e0770de24783a4077e69413ed6b0d4b43ba5e5574728348c04e1241a8d07f659
  Stored in directory: /root/.cache/pip/wheels/91/58/42/16adbc26648dbdeac90238a8a3

## Setup Model


In [ ]:
import os
import requests 
from jax.config import config

# colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
# url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
# requests.post(url)

# # The following is required to use TPU Driver as JAX's backend.
# config.FLAGS.jax_xla_backend = "tpu_driver"
# config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

In [ ]:
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,
  "early_cast": True, #NMB: Add this
  "seq": 2048,
  "cores_per_replica": 1, #NMB: Running on 1 GPU
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]

params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

#mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica) # NMB: Remove this
#devices = np.array(jax.devices()).reshape(mesh_shape) # NMB: Remove this
devices = np.array([jax.devices()[0]]).reshape((1, 1))
maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

Here we create the network and load the parameters from the downloaded files. Expect this to take around 5 minutes.

In [ ]:
#total_batch = per_replica_batch * jax.device_count() // cores_per_replica #NMB: Remove this

network = CausalTransformer(params)

# here we load a checkpoint which was written with 8 shards into 1 shard
#network.state = read_ckpt(network.state, "step_383500/", devices.shape[1]) #NMB: Remove this
network.state = read_ckpt(network.state, "step_383500/", 8, shards_out=cores_per_replica) #NMB: Add this

#network.state = network.move_xmap(network.state, np.zeros(cores_per_replica)) #NMB: Remove this

/usr/local/lib/python3.6/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (1, 2)
in shape (1, 2048)
dp 1
mp 1


## Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when changed).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
# NMB: Replace total_batch with per_replica_batch

start = time.time()

def infer(context, top_p=0.9, temp=1.0, gen_len=512):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * per_replica_batch)
    length = np.ones(per_replica_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(per_replica_batch) * top_p, "temp": np.ones(per_replica_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(tokenizer.decode(o))

    print(f"completion done in {time.time() - start:06}s")
    return samples

print(infer("EleutherAI is")[0])

In [ ]:
#@title  { form-width: "300px" }
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}

context = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

print(infer(top_p=top_p, temp=temp, gen_len=512, context=context)[0])